## 1. 建立並保存 ONNX 模型檔案
以下是一個使用 TensorFlow 建立鳶尾花（Iris）分類模型並將其導出為 ONNX 格式的範例。該模型使用簡單的全連接層來進行分類，並轉換為 ONNX 格式，方便在 TVM 或其他 ONNX 支持的推理引擎上運行。

### 1.1 安裝必要的套件
如果尚未安裝 tensorflow 和 tf2onnx，可以使用以下命令安裝：



In [ ]:
!pip install tensorflow tf2onnx

### 1.2 建立並訓練 TensorFlow 模型
以下程式碼將建立一個簡單的神經網絡來分類鳶尾花數據集，並將其導出為 ONNX 格式。

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# 載入鳶尾花資料集
iris = load_iris()
X = iris.data.astype(np.float32)
y = iris.target

# 分割資料集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 建立模型
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(4,)),  # 4 個特徵
    tf.keras.layers.Dense(10, activation='relu'),  # 隱藏層
    tf.keras.layers.Dense(3, activation='softmax') # 輸出層，3 個分類
])

# 編譯模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練模型
model.fit(X_train, y_train, epochs=50, batch_size=5, verbose=0)

# 評估模型
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"模型準確率: {accuracy:.2f}")

2024-11-13 11:47:00.357040: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-13 11:47:00.397227: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-13 11:47:00.397246: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-13 11:47:00.397274: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-13 11:47:00.404958: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-13 11:47:00.405555: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

模型準確率: 1.00


### 1.3 將模型轉換為 ONNX 格式
使用 tf2onnx 將訓練好的 TensorFlow 模型轉換為 ONNX 格式：

In [1]:
import tf2onnx

# 將 Keras 模型轉換為 ONNX 格式
spec = (tf.TensorSpec((None, 4), tf.float32, name="float_input"),)  # 定義輸入規範
output_path = "tf_model.onnx"  # 輸出 ONNX 模型的路徑

# 轉換模型
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13)
with open(output_path, "wb") as f:
    f.write(model_proto.SerializeToString())

print(f"ONNX 模型已保存至 {output_path}")

2024-11-13 16:30:12.466954: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-13 16:30:12.504919: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-13 16:30:12.504939: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-13 16:30:12.504963: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-13 16:30:12.512059: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-13 16:30:12.512504: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

NameError: name 'tf' is not defined

In [3]:
import onnxruntime as ort
import numpy as np

# 加載 ONNX 模型
session = ort.InferenceSession('tf_model.onnx')

# 準備輸入資料
input_name = session.get_inputs()[0].name
input_data = np.array([[6.3, 3.3, 6.0, 2.5]], dtype=np.float32)

# 進行推理
pred_onnx = session.run(None, {input_name: input_data})

# 輸出預測結果
print(pred_onnx)

[array([[5.2434858e-04, 8.7534554e-02, 9.1194111e-01]], dtype=float32)]


## 2. TVM 進行編譯產生 C
### 2.1 轉換模型為 Relay 格式
在 TVM 中，我們可以使用 ONNX 格式轉換模型，然後導入 TVM 中。如果使用者已有其他框架的模型（如 TensorFlow、Keras、PyTorch），可相應轉換為 Relay 支持的格式。

In [ ]:
!export TVM_HOME=/home/jovyan/project/ONNX-MLIR/tvm
!export PYTHONPATH=$TVM_HOME/python:$PYTHONPATH

In [ ]:
!TVM_LIBRARY_PATH=/home/jovyan/project/ONNX-MLIR/tvm/build python3 run.py

In [1]:
import tvm
print(tvm.__file__)

[20:52:22] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[20:52:22] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[20:52:22] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`


/home/jovyan/project/ONNX-MLIR/tvm/python/tvm/__init__.py


In [2]:
!python -c "import tvm; print(tvm.__file__)"

[17:28:34] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[17:28:34] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[17:28:34] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
/home/jovyan/project/ONNX-MLIR/tvm/python/tvm/__init__.py


In [2]:
import onnx
import skl2onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType


# 導入 ONNX 模型至 TVM Relay
import tvm.relay as relay

onnx_model = onnx.load("tf_model.onnx")
shape_dict = {"float_input": (1, 4)}
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

###  2.2 設置 microTVM 和生成 C 代碼
使用 microTVM 的 AOT（Ahead-of-Time）執行器來生成適合嵌入式設備的 C 代碼。

In [6]:
!TVM_LIBRARY_PATH=/home/jovyan/project/ONNX-MLIR/tvm/build python3 run.py

[17:29:40] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[17:29:40] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[17:29:40] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.
Traceback (most recent call last):
  File "/home/jovyan/project/ONNX-MLIR/tvm-tutorial/run.py", line 23, in <module>
    lowered_module.get_lib().export_library("iris_model.c", tvm.micro.export_model_library_format)
AttributeError: 

In [5]:
import tvm
# from tvm import relay
# import tvm.micro

# 定義目標硬體平台 (這裡以 `c` 為主，代表原生 C 代碼)
target = tvm.target.target.micro("host")

# 設定編譯選項
with tvm.transform.PassContext(opt_level=3):
    lowered_module = relay.build(mod, target='c', params=params, executor=tvm.relay.backend.Executor("aot"))

# 將生成的模型保存為 C 源碼
lowered_module.get_lib().export_library("iris_model.c")

RuntimeError: Compilation error:
/tmp/tmp5y2hzsyt/lib0.c: In function ‘int32_t tvmgen_default_fused_nn_contrib_dense_pack_add(void*, int32_t*, int32_t, void*, int32_t*, void*)’:
/tmp/tmp5y2hzsyt/lib0.c:95:3: error: ‘float3’ was not declared in this scope; did you mean ‘float’?
   95 |   float3 compute_global[1];
      |   ^~~~~~
      |   float
/tmp/tmp5y2hzsyt/lib0.c:96:3: error: ‘compute_global’ was not declared in this scope
   96 |   compute_global[0] = ((float3)(0.000000e+00f, 0.000000e+00f, 0.000000e+00f));
      |   ^~~~~~~~~~~~~~
/tmp/tmp5y2hzsyt/lib0.c:98:144: error: expected primary-expression before ‘)’ token
   98 |     compute_global[0] = (compute_global[0] + (((float3)(((float*)p0_1)[k_outer], ((float*)p0_1)[k_outer], ((float*)p0_1)[k_outer])) * *(float3*)(((float*)fused_constant_1) + (k_outer * 3))));
      |                                                                                                                                                ^
/tmp/tmp5y2hzsyt/li

## microTVM

In [ ]:
import tvm
from tvm import relay
from tvm.contrib import utils
from tvm.relay.backend import Runtime
import onnx

# 載入 ONNX 模型
onnx_model = onnx.load("tf_model.onnx")

# 將 ONNX 模型轉換為 Relay 模型
input_name = 'float_input'  # 輸入名稱依據您的 ONNX 模型
shape_dict = {input_name: (1, 4)}
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

# 設置目標架構並啟用 C runtime
target = tvm.target.Target("llvm -system-lib -mtriple=x86_64-linux-gnu")
runtime = Runtime("c")

with tvm.transform.PassContext(opt_level=3):
    # 編譯模型，指定 runtime=runtime 並啟用 system-lib
    lib = relay.build(mod, target=target, params=params, runtime=runtime)

# 將 C 代碼導出為 model.c
lib.export_library("model.c", cc="gcc")


In [ ]:
!ls

In [ ]:
!tar -xvf ./module.tar